In [4]:
# Install dependencies (run once per environment)
%pip install -q dspy python-dotenv openai>=1.40.0

zsh:1: 1.40.0 not found
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Basic imports and environment setup
import os
import dspy
from dotenv import load_dotenv
from dspy import History

# Load API keys from .env
load_dotenv()

# OpenAI SDK client
from openai import OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Configure model provider (OpenAI-only, per LangGraph agent)
lm = dspy.LM("openai/gpt-5-mini", api_key=OPENAI_API_KEY, temperature=1, max_tokens=16000)

dspy.configure(lm=lm)

print("DSPy configured for Deep Research agent.")


DSPy configured for Deep Research agent.


In [6]:
# Utils: helpers
from datetime import datetime


def get_today_str() -> str:
    now = datetime.now()
    return f"{now:%a} {now:%b} {now.day}, {now:%Y}"


In [7]:
# Override openai_search to use OpenAI Responses API web_search
from typing import List, Literal


def openai_search(queries: List[str], max_results: int = 5, topic: Literal["general", "news"] = "general") -> str:
    """Generate research using OpenAI Responses API with native web_search.

    Calls OpenAI `responses.create` with tools=[{"type": "web_search"}] for each query.
    """
    if not isinstance(queries, list) or not queries:
        return "No research results could be generated. Please provide queries."

    results: List[str] = []
    for q in queries[:max_results]:
        try:
            if topic == "news":
                user_input = (
                    f"For today's news, what's the latest on: \"{q}\"? "
                    "Summarize key facts, timeline, status, implications, and include sources."
                )
            else:
                user_input = (
                    f"Please research: \"{q}\". "
                    "Provide key facts, context, implications, examples, and include sources."
                )

            resp = client.responses.create(
                model="gpt-5-mini",
                tools=[{"type": "web_search"}],
                input=user_input,
            )

            content = getattr(resp, "output_text", None)
            if not content:
                content = str(resp)

            results.append(f"--- RESEARCH RESULT: {q} ---\n{content}\n")
        except Exception as e:
            results.append(f"--- RESEARCH RESULT: {q} ---\nError calling OpenAI Responses API: {e}\n")

    return (f"OpenAI Research Results ({topic} focus):\n\n" + "\n\n".join(results)).strip()


In [8]:
openai_search(queries=["what is the weather like today in London"], max_results = 5, topic = "news")

'OpenAI Research Results (news focus):\n\n--- RESEARCH RESULT: what is the weather like today in London ---\nQuick update for London — today is Saturday, October 4, 2025.\n\nKey facts\n- Current conditions: light rain, about 14°C (57°F).   \n- Overall trend: showers/brief rain this afternoon, cloudiness through early evening, clearing to partly/mostly clear overnight. Temperatures fall from mid-teens °C this afternoon to about 11°C overnight. \n\nTimeline (local London time)\n- 15:00 — Showers, ~15°C.  \n- 16:00 — Cloudy, ~15°C.  \n- 17:00 — Intermittent clouds, ~14°C.  \n- 18:00 — Partly sunny, ~14°C.  \n- 19:00 — Mostly clear, ~13°C.  \n- 20:00 — Partly cloudy, ~13°C.  \n- 21:00 — Intermittent clouds, ~12°C.  \n- 22:00 — Mostly cloudy, ~12°C.  \n- 23:00 — Intermittent clouds, ~12°C.  \n- 00:00–02:00 — Becoming clear, down to ~11°C. \n\nStatus\n- Short-term: Light rain currently with scattered showers expected to taper through late afternoon. Clearing is likely by evening with dry, co

In [9]:
# ReAct Signature for Deep Research

class ResearchReActSignature(dspy.Signature):
    """
    You are a deep research assistant. Use tools to gather information (openai_search for research).
    Keep searches focused, reflect after searches, and finish with a concise, well-structured research answer.

    You can call tools: openai_search.
    When finished, produce:
    - `action`: the primary tool used (one of: openai_search, answer_direct)
    - `tool_result`: the most relevant tool output you used (may be empty for answer_direct)
    - `answer`: the final research answer/report
    Keep responses clear and professional.
    """
    user_message: str = dspy.InputField(description="The user's research request")
    history: dspy.History = dspy.InputField(description="Conversation history")

    reasoning: str = dspy.OutputField(description="Brief plan and justification")
    action: str = dspy.OutputField(description="Chosen action/tool")
    tool_result: str = dspy.OutputField(description="Tool output used to answer")
    answer: str = dspy.OutputField(description="Final research answer")


In [10]:
# Module-based ReAct agent for chat continuity

class DeepResearchAgent(dspy.Module):
    def __init__(self, max_iters: int = 5):
        super().__init__()
        self.conversation_history = dspy.History(messages=[])
        self.research = dspy.ReAct(
            ResearchReActSignature,
            tools=[openai_search],
            max_iters=max_iters,
        )

    def forward(self, user_message: str):
        # Append user message to internal history
        self.conversation_history.messages.append({"role": "user", "content": user_message})
        # Run ReAct with internal history
        result = self.research(user_message=user_message, history=self.conversation_history)
        # Append assistant answer back to history
        answer = getattr(result, "answer", "")
        if isinstance(answer, str) and answer.strip():
            self.conversation_history.messages.append({"role": "assistant", "content": answer})
        return result



In [11]:
# Agent instance

agent = DeepResearchAgent(max_iters=5)



In [12]:
# Examples / smoke tests (Module-based ReAct)

print("\n--- Example 1 ---")
resp = agent(user_message="Give me a concise overview of Rust ownership vs borrow checker with examples.")
print({
    "answer": getattr(resp, "answer", ""),
    "action": getattr(resp, "action", ""),
    "tool_result": getattr(resp, "tool_result", ""),
})

print("\n--- Example 2 ---")
resp2 = agent(user_message="Summarize the latest approaches to structured reasoning in LLMs.")
print({
    "answer": getattr(resp2, "answer", ""),
    "action": getattr(resp2, "action", ""),
    "tool_result": getattr(resp2, "tool_result", ""),
})



--- Example 1 ---
{'answer': 'Concise overview: Ownership vs Borrow Checker (with short examples and tips)\n\n1) Core idea (high level)\n- Ownership: Every value in Rust has a single owner (a variable). When the owner goes out of scope, the value is dropped (automatic memory management without GC).\n- Borrowing: You can lend access to a value via references (&T or &mut T) instead of transferring ownership.\n- Borrow checker: A compiler phase that enforces rules about references to ensure memory safety (no use-after-free, no data races).\n\n2) Basic ownership rules\n- Move semantics: Assigning or passing a non-Copy type (e.g., String, Vec<T>) moves ownership.\n  Example:\n  let s1 = String::from("hello");\n  let s2 = s1; // s1 moved, s2 now owns the String\n  // using s1 here would be a compile error\n\n- Copy types (primitive integers, bool, char, types that implement Copy) are copied, not moved:\n  let x = 5;\n  let y = x; // both x and y usable\n\n- Clone to duplicate heap data expl

In [14]:
lm.inspect_history(n=3)





[2025-10-04T14:25:55.878094]

System message:

Your input fields are:
1. `user_message` (str): The user's research request
2. `history` (History): Conversation history
3. `trajectory` (str):
Your output fields are:
1. `next_thought` (str): 
2. `next_tool_name` (Literal['openai_search', 'finish']): 
3. `next_tool_args` (dict[str, Any]):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## user_message ## ]]
{user_message}

[[ ## history ## ]]
{history}

[[ ## trajectory ## ]]
{trajectory}

[[ ## next_thought ## ]]
{next_thought}

[[ ## next_tool_name ## ]]
{next_tool_name}        # note: the value you produce must exactly match (no extra characters) one of: openai_search; finish

[[ ## next_tool_args ## ]]
{next_tool_args}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "additionalProperties": true}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a dee